In [7]:
#data analysis
import pandas as pd
import numpy as np

#data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#models
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet, LinearRegression, Ridge

#model validation and preprocessing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split, RepeatedKFold
#from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler, PolynomialFeatures
from tqdm import tqdm
from scipy.stats import randint, uniform

solar = pd.read_csv('rhessi.csv')


In [8]:


'''
Creates intensity target attribute through avg of energy.kev * duration.s
- takes a dataframe and returns dataframe with added 'intensity' column
'''
def addIntensity(data):
    avg_energy = (data['energy.kev.i'] + data['energy.kev.f'])/2
    data['intensity'] = avg_energy * data['duration.s']
    return data

#method 1's:

M1 = XGBRegressor(learning_rate= 0.248, max_depth= 10, n_estimators= 546)

y = solar[["total.counts"]]
X = solar[['duration.s', 'peak.c/s', 'x.pos.asec',
           'y.pos.asec', 'radial', 'energy.kev.i',
           'energy.kev.f', 'day','month','year','active.region.ar']]

M1.fit(X, y)

#method 2's:
solar = addIntensity(solar)

M2 = RandomForestRegressor()

y = solar[["intensity"]]
X = solar[['day','month','year','active.region.ar', 'peak.c/s', 'total.counts', 'x.pos.asec', 'y.pos.asec', 'radial']]

M2.fit(X, y)





C:\Users\Joseph\AppData\Local\Temp\ipykernel_8604\1804467434.py:29: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  M2.fit(X, y)


RandomForestRegressor()

In [9]:
#solar flare data from 2004 and 2005
solar_2004_2005 = solar[(solar['year'] == 2004) | (solar['year'] == 2005)]

#solar flare data from 2015 and 2016
solar_2015_2016 = solar[(solar['year'] == 2015) | (solar['year'] == 2016)]

In [10]:
#Make Batches Function
def make_batches(data, start_year, end_year):
    batches = []
    for start_month in range(1, 12, 2):
        batch = data[
            (data['year'] == start_year) & (data['month'].between(start_month, start_month + 3))]
        if len(batches) == 5:
            batch = data[
                ((data['year'] == start_year) & data['month'].between(start_month, start_month + 3)) |
                ((data['year'] == end_year) & data['month'].between(1, 2))
            ]
        batches.append(batch)

    for start_month in range(1, 12, 2):
        batch = data[
            (data['year'] == end_year) & data['month'].between(start_month, start_month + 3)]
        if len(batches) <= 10:
            batches.append(batch)
    return batches

In [11]:
batches2004_2005 = make_batches(solar_2004_2005, 2004, 2005)
batches2015_2016 = make_batches(solar_2015_2016, 2015, 2016)

In [29]:
# Moreover, compute basic statistics (e.g. various counts and averages) for Set 1 and Set 2 and compare those. 

solar_2004_2005['intensity'].describe()

count    1.750600e+04
mean     7.418507e+03
std      3.129340e+04
min      7.200000e+01
25%      1.980000e+03
50%      3.528000e+03
75%      6.882000e+03
max      2.200000e+06
Name: intensity, dtype: float64

In [27]:
solar_2015_2016['intensity'].describe()

count    1.077900e+04
mean     9.760980e+03
std      3.012329e+04
min      7.200000e+01
25%      2.088000e+03
50%      3.816000e+03
75%      7.755000e+03
max      1.337600e+06
Name: intensity, dtype: float64

In [12]:
# Batches 0 and 10 for the year 2004-2005
# Naming convention should follow: batchN_methodN_2004_2005
# batch0 1+2+3+4 (also known as batch 1)
batch0_method1_2004_2005 = batches2004_2005[0][['duration.s', 'peak.c/s', 'x.pos.asec',
                                'y.pos.asec', 'radial', 'energy.kev.i',
                                'energy.kev.f', 'day','month','year','active.region.ar']]

batch0_method2_2004_2005 = batches2004_2005[0][['day','month','year','active.region.ar', 'peak.c/s',
                            'total.counts', 'x.pos.asec', 'y.pos.asec', 'radial']]

# batch10 21+22+23+24 (also known as batch 11)
batch10_method1_2004_2005 = batches2004_2005[10][['duration.s', 'peak.c/s', 'x.pos.asec',
                                'y.pos.asec', 'radial', 'energy.kev.i',
                                'energy.kev.f', 'day','month','year','active.region.ar']]

batch10_method2_2004_2005 = batches2004_2005[10][['day','month','year','active.region.ar', 'peak.c/s',
                            'total.counts', 'x.pos.asec', 'y.pos.asec', 'radial']]

# Batch 0 Method 1 2004_2005
batch0_method1_y_pred_2004_2005 = M1.predict(batch0_method1_2004_2005)
batch0_method1_2004_2005['intensity_pred'] = batch0_method1_y_pred_2004_2005

# Batch 0 Method 2 2004_2005
batch0_method2_y_pred_2004_2005 = M2.predict(batch0_method2_2004_2005)
batch0_method2_2004_2005['intensity_pred'] = batch0_method2_y_pred_2004_2005

# Batch 10 Method 1 2004_2005
batch10_method1_y_pred_2004_2005 = M1.predict(batch10_method1_2004_2005)
batch10_method1_2004_2005['intensity_pred'] = batch10_method1_y_pred_2004_2005

# Batch 10 Method 2 2004_2005
batch10_method2_y_pred_2004_2005 = M2.predict(batch10_method2_2004_2005)
batch10_method2_2004_2005['intensity_pred'] = batch10_method2_y_pred_2004_2005

C:\Users\Joseph\AppData\Local\Temp\ipykernel_8604\4006077368.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch0_method1_2004_2005['intensity_pred'] = batch0_method1_y_pred_2004_2005
C:\Users\Joseph\AppData\Local\Temp\ipykernel_8604\4006077368.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch0_method2_2004_2005['intensity_pred'] = batch0_method2_y_pred_2004_2005
C:\Users\Joseph\AppData\Local\Temp\ipykernel_8604\4006077368.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [13]:
# Batches 0 and 10 for the year 2015-2016
# batch0
batch0_method1_2015_2016 = batches2015_2016[0][['duration.s', 'peak.c/s', 'x.pos.asec',
                                'y.pos.asec', 'radial', 'energy.kev.i',
                                'energy.kev.f', 'day','month','year','active.region.ar']]

batch0_method2_2015_2016 = batches2015_2016[0][['day','month','year','active.region.ar', 'peak.c/s',
                            'total.counts', 'x.pos.asec', 'y.pos.asec', 'radial']]

# batch10
batch10_method1_2015_2016 = batches2015_2016[10][['duration.s', 'peak.c/s', 'x.pos.asec',
                                'y.pos.asec', 'radial', 'energy.kev.i',
                                'energy.kev.f', 'day','month','year','active.region.ar']]

batch10_method2_2015_2016 = batches2015_2016[10][['day','month','year','active.region.ar', 'peak.c/s',
                            'total.counts', 'x.pos.asec', 'y.pos.asec', 'radial']]

# Batch 0 Method 1 2015_2016
batch0_method1_y_pred_2015_2016 = M1.predict(batch0_method1_2015_2016)
batch0_method1_2015_2016['intensity_pred'] = batch0_method1_y_pred_2015_2016

# Batch 0 Method 2 2015_2016
batch0_method2_y_pred_2015_2016= M2.predict(batch0_method2_2015_2016)
batch0_method2_2015_2016['intensity_pred'] = batch0_method2_y_pred_2015_2016

# Batch 10 Method 1 2015_2016
batch10_method1_y_pred_2015_2016 = M1.predict(batch10_method1_2015_2016)
batch10_method1_2015_2016['intensity_pred'] = batch10_method1_y_pred_2015_2016

# Batch 10 Method 2 2015_2016
batch10_method2_y_pred_2015_2016 = M2.predict(batch10_method2_2015_2016)
batch10_method2_2015_2016['intensity_pred'] = batch10_method2_y_pred_2015_2016

C:\Users\Joseph\AppData\Local\Temp\ipykernel_8604\4072231292.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch0_method1_2015_2016['intensity_pred'] = batch0_method1_y_pred_2015_2016
C:\Users\Joseph\AppData\Local\Temp\ipykernel_8604\4072231292.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch0_method2_2015_2016['intensity_pred'] = batch0_method2_y_pred_2015_2016
C:\Users\Joseph\AppData\Local\Temp\ipykernel_8604\4072231292.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [6]:
#cannot use features used to create 'intensity' solar[['energy.kev.i', 'energy.kev.f', 'duration.s']]
#Method 2: first creating the intensity variable (target_variable)

'''
currently creates intensity target parameter through avg of energy.kev * duration.s
'''
def addIntensity(data):
    avg_energy = (data['energy.kev.i'] + data['energy.kev.f'])/2
    data['intensity'] = avg_energy * data['duration.s']
    return data


solar_2 = addIntensity(solar)
"""
#method 1's:
y = solar_2[["total.counts"]]
X = solar_2[['duration.s', 'peak.c/s', 'x.pos.asec', 
           'y.pos.asec', 'radial', 'energy.kev.i', 
           'energy.kev.f', 'day','month','year','active.region.ar']]
"""

#method 2's:
y = solar_2[["intensity"]]
X = solar_2[['day','month','year','active.region.ar', 'peak.c/s', 'total.counts', 'x.pos.asec', 'y.pos.asec', 'radial']]


X_train, X_test, y_train, y_test = train_test_split(X, y,test_size =0.2, random_state = 42)

# Results (M1):

XGBRegressor(learning_rate= 0.248, max_depth= 10, n_estimators= 546)
- Mean MAE: 138029.805 (17625.737)

RandomForestRegressor()
- Mean MAE: 139686.422 (17620.332)

Pipeline( [('poly' , PolynomialFeatures(degree = 2, include_bias = False, interaction_only = False)),                  ('lin_r', LinearRegression()) ])
- Mean MAE: 282348.380 (253154.974)

SVR()
- Mean MAE: 347294.496 (35389.136)

slr = Pipeline([('scaler', RobustScaler()),('svr',SVR(max_iter = 100000))])
- Mean MAE: 347933.764 (35413.271)

Pipeline([('scaler', RobustScaler()), ('linear', LinearRegression(fit_intercept = False))])
- Mean MAE: 356138.574 (17816.858)

Pipeline(steps=[('robust', RobustScaler()),
                ('ridge',
                 Ridge(alpha=9.391053524878311, tol=0.000841380674450156))])
- Mean MAE: 402379.564 (16763.954)

LinearRegression()
- Mean MAE: 402429.099 (16769.803)

ridge = Ridge(alpha=8.501992226787863, tol=0.0007708353644612364)
- Mean MAE: 402429.088 (16769.796)



# Results (M2):

XGBRegressor(learning_rate= 0.0597, max_depth= 4, n_estimators= 229)
- Mean MAE: 3971.186 (170.351)

RandomForestRegressor()
- Mean MAE: 4162.393 (193.812)

Pipeline([('poly' , PolynomialFeatures(degree = 3, include_bias = False, interaction_only = True)), ('lin_r', LinearRegression())])
- Mean MAE: 3157.079 (102.188)

LinearRegression()
- Mean MAE: 3262.677 (100.384)

Ridge(alpha=9.739625063533492, tol=7.842778538788441e-05)
- Mean MAE: 3262.677 (100.384)

Pipeline(steps=[('standard', StandardScaler()),
                ('ridge',
                 Ridge(alpha=1.157737190630861, tol=0.0008946473923448764))])
- Mean MAE: 3262.680 (100.384)

In [12]:
'''
We make the mean_absolute_error negative because scikit-learn assumes a higher number is better for scoring.
That's why we take the absolute value of our score at the end.
A lower MAE is better! 
'''
def cv(model):
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate model
    scores = cross_val_score(model, X_train, y_train, 
                             scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    
    #for cross_val being used:
    scores = np.absolute(scores)
    print(scores)
    print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )
    


"""
The different search algo's benefits:

RandomizedSearchCV
    + less computationally expensive than GridSearchCV
    - not as robust(?) as GridSearchCV

GridSearchCV
    - computationally expensive
    - does not do well with large param list
    
HalvingGridSearchCV
    + less computationally expensive
    + does some battle royal stuff to narrow search pool
    - for complex models and param lists
"""
def grid(model, params):
    grid_search = RandomizedSearchCV(model, params, cv=5, scoring='neg_mean_absolute_error',
                                     verbose=10, n_jobs=10, return_train_score=True)
    grid_search.fit(X_train, y_train)
    print("\n The best estimator:\n", grid_search.best_estimator_)
    print("\n The best score:\n", grid_search.best_score_)
    print("\n The best parameters:\n", grid_search.best_params_)
    
    

"""
XGBoost is known as one of the strongest models, need to become like thanos and harness this stone

XGBoost Notes: 
- when we changed the scoring within our gridsearch to 'neg_mean_absolute_error' the hyperparameters performed better

RandomForest Notes:
- base model performed just as well as optimized XGBoost, ffs
- RandomForest after gridSearch performed worse than base model

polynomial regression:
- takes up a lot of memory (failed some fits during hyperparameter tuning bc of this)
- need to kernel trick features (feature space mapping)


scaling vs. centering
- scaling is when we want to transform our values to a specific range (everything's got similar magnitudes)
- centering is shifting values so they have a mean of 0 (subtract mean of feature from every value)

linear Regression:
- dogshit without scaling (probably)
- linear Regression also performed essentially the same with and without scaling, this brought to my attention 
the fact that scikit-learn has it's own internal normalization step during training (go figure)
- for M1, when fit_intercept = False, standardScaler pipeline shit the bed, but RobustScaler did better than 
base linear Regression & base Robust linear Regression (interesting, wonder if has to do with the centering done by
RobustScaler & it's ability to handle data with outliers)

ridge Regression:
- ridge regression is less sensitive to magnitude (both types of scaled & unscaled data got same result)
- performed better on M1 than M2, standardScaler performed better than RobustScaler on M1 as well


RobustScaler vs StandardScaler
- Robust uses median & IQR
- Standard uses mean & standard deviation
- Robust is good for outlier heavy dataset, Standard is not

Support Vector Machine:
- So goddamn computationally expensive, it's wild
- Oddly enough, using RobustScaler on the data resulted in a worse MAE

Thoughts on results:
- The dataset seems to be complex (go figure dumbass), the performance of the linear model and it's variations
does not compare to the tree algorithms
"""
#xg= XGBRegressor(learning_rate= 0.248, max_depth= 10, n_estimators= 546)
#Mean MAE: 138029.805 (17625.737)


xgr = Pipeline([('poly' , PolynomialFeatures(degree = 3, include_bias = False, interaction_only = True)), ('lin_r', LinearRegression())])

'''
example of the params_list for the XGBRegressor() model
used for RandomizedSearchCV
'''
xg_params = {
    'n_estimators' : randint(100,1201),
    'max_depth' : randint(1,11),
    'learning_rate' : uniform(0.001, 0.3)
}

#Next task: 

slr = Pipeline([('scaler', RobustScaler()),('svr',SVR(max_iter = 100000))])
#main:

cv(xgr)

[4849.42691497 5545.17238524 4655.90128572 5031.38925448 5280.22936411
 4877.37605448 4721.62592599 5005.0724105  5084.90464842 5589.01963599
 5091.56428457 4995.27169878 4906.52154967 5158.38121743 5374.0170681
 5170.12071312 4636.55675461 4740.67858758 5074.1103134  4890.20900364
 4755.47401167 5015.79602125 5321.70589645 4738.58029261 5351.96264504
 4900.01148511 5053.72903121 5091.83810483 4998.89795772 4973.79452312]
Mean MAE: 5029.311 (241.139)


In [ ]:
solar_2.describe()

### Everything after this point is just experimentation:

In [ ]:
#DecisionTreeRegressor Pipelines
#we are going to score it in different ways 
treePipe = Pipeline([('regressor', RandomForestRegressor(random_state=0, verbose=3, n_jobs=10))])
tree_params = {
    'regressor__criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],  # Splitting criterion
    'regressor__max_depth': [None, 10, 20, 30, 40],  # Maximum depth of the tree
    'regressor__min_samples_leaf': [1, 2, 4], # Minimum samples required at a leaf node
}

grid_search = GridSearchCV(treePipe, tree_params,
                           cv=5, verbose=10, 
                           refit=True)

#results: friedman_mse, regression max_depth = 40, min_samples leaf=4
#results 2.0 = squared error, regressor_max_depth = None, min_samples leaf = 1
#things we learned: gridsearchcv needs to handeled with care
grid_search.fit(X, np.ravel(y))

In [ ]:
forest = RandomForestRegressor(n_jobs=5, 
                      verbose=3, criterion='friedman_mse', 
                      max_depth =40, min_samples_leaf=4)
#with random_state = 0, score is 0.95
#without setting random_state obtained a result similar 
scores = cross_val_score(forest, X, y, n_jobs = 1)
scores.mean()

In [ ]:
svr_params = {
    'svr__C': [0.1, 1, 10, 100],
    'svr__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'svr__gamma': [0.01, 0.1, 1, 'scale', 'auto'],
}
svr_pipe = Pipeline([('scaler', StandardScaler()), ('svr', SVR())])
#Notes:
#without scaled data, score's were very large or non-existent, going to experiment now with max_iter and scaled data
grid_search = RandomizedSearchCV(svr_pipe, svr_params, cv=5,
 n_iter = 5,verbose=10,
 n_jobs=-1, return_train_score=True)
grid_search.fit(X, np.ravel(y))
print("\n The best estimator:\n", grid_search.best_estimator_)
print("\n The best score:\n", grid_search.best_score_)
print("\n The best parameters:\n", grid_search.best_params_)

In [ ]:
#GradientBoostingRegressor is not looking good for method 2, we are going to try it on method 1 




y = solar_2[["intensity"]]
X = solar_2[['day','month','year','active.region.ar', 'peak.c/s', 'total.counts', 'x.pos.asec', 'y.pos.asec', 'radial']]
gb = GradientBoostingRegressor(random_state=2)

initial_params = {}
gs = GridSearchCV(estimator = gb, param_grid = initial_params,n_jobs=10, verbose = 10)
gs.fit(X, np.ravel(y))
print("\n The best estimator:\n", gs.best_estimator_)
print("\n The best score:\n", gs.best_score_)
print("\n The best parameters:\n", gs.best_params_)

In [ ]:
y = solar[["total.counts"]]
X = solar[['duration.s', 'peak.c/s', 'x.pos.asec', 'y.pos.asec', 'radial', 'energy.kev.i', 'energy.kev.f']]
#gb = GradientBoostingRegressor(random_state=2, learning_rate = 0.2, n_estimators=40, min_samples_split = 10)
gb = XGBRegressor(random_state=2)
param_grid = {'learning_rate':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]}
gs = GridSearchCV(estimator = gb, param_grid = initial_params,
                  n_jobs=10, verbose = 10)
gs.fit(X, y)
print("\n The best estimator:\n", gs.best_estimator_)
print("\n The best score:\n", gs.best_score_)
print("\n The best parameters:\n", gs.best_params_)

In [ ]:
gb_params = {'gradient__min_samples_split': [0.01, 0.1, 1, 'scale', 'auto'], 
              'gradient_warm_start' : [False, True],
              'gradient_learning_rate' : [0.05, 0.1, 0.15, 0.2]}
#Notes:
#svr's were not able to capture the relationship
grid_search = RandomizedSearchCV(gb_pipe, gb_params, cv=5,
 n_iter = 5,verbose=10, random_state=42,
 n_jobs=-1, return_train_score=True)
grid_search.fit(X, np.ravel(y))
print("\n The best estimator:\n", grid_search.best_estimator_)
print("\n The best score:\n", grid_search.best_score_)
print("\n The best parameters:\n", grid_search.best_params_)

In [ ]:
grid_search = RandomizedSearchCV(svr_pipe, svr_params, cv=5,
 n_iter = 5,verbose=10, random_state=42,
 n_jobs=10, return_train_score=True)
grid_search.fit(X, np.ravel(y))
print("\n The best estimator:\n", grid_search.best_estimator_)
print("\n The best score:\n", grid_search.best_score_)
print("\n The best parameters:\n", grid_search.best_params_)

In [22]:
solar_2.describe()

,duration.s,peak.c/s,total.counts,x.pos.asec,y.pos.asec,radial,active.region.ar,year,month,day,energy.kev.i,energy.kev.f,intensity
count,108182.000000,108182.000000,1.081820e+05,108182.000000,108182.000000,108182.000000,108182.000000,108182.000000,108182.000000,108182.000000,108182.000000,108182.000000,1.081820e+05
mean,494.275647,210.735363,3.735130e+05,0.943780,-34.201485,713.441404,1056.193535,2009.092927,6.672080,15.588203,7.930469,16.418323,4.718839e+03
std,431.102410,852.139223,3.114586e+06,708.603208,264.625164,253.929243,1371.022947,4.839022,3.454303,8.794025,22.561547,70.015105,1.271962e+04
min,8.000000,0.000000,8.000000e+00,-1007.000000,-998.000000,0.000000,0.000000,2002.000000,1.000000,1.000000,6.000000,12.000000,4.800000e+01
25%,212.000000,28.000000,2.263200e+04,-731.000000,-254.000000,519.000000,50.000000,2004.000000,4.000000,8.000000,6.000000,12.000000,1.320000e+03
50%,368.000000,52.000000,5.695200e+04,-5.000000,-101.000000,785.000000,758.000000,2011.000000,7.000000,15.000000,6.000000,12.000000,2.376000e+03
75%,628.000000,136.000000,1.722240e+05,736.000000,206.000000,948.000000,1620.000000,2013.000000,10.000000,23.000000,6.000000,12.000000,4.632000e+03
max,4444.000000,113156.000000,4.355501e+08,1005.000000,1012.000000,1015.000000,9999.000000,2018.000000,12.000000,31.000000,7000.000000,20000.000000,1.200000e+06
